In [1]:
import mlflow
from sentence_transformers import SentenceTransformer
from mlflow.models.signature import infer_signature


MLFLOW_TRACKING_URI = "http://localhost:5000/"
MODEL_NAME = "all-MiniLM-L12-v2"
EXPERIMENT_NAME = "deployment-ready-embedding-model"
ALIAS = "champion"

# local mlflow service is running in a container
# https://github.com/mlflow/mlflow/tree/master/docker-compose
mlflow.set_tracking_uri(uri=MLFLOW_TRACKING_URI)

/home/joshuale/miniconda3/envs/local-rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def log_deployment_ready_model():
    """Create a production-ready semantic search model."""
    mlflow.set_experiment(EXPERIMENT_NAME)
    with mlflow.start_run() as run:
        print("Started run with info:")
        print(run.info)
        model = SentenceTransformer(MODEL_NAME)

        sample_input = ["input text"]
        sample_output = model.encode(sample_input)
        signature = infer_signature(model_input=sample_input, model_output=sample_output)

        model_info = mlflow.sentence_transformers.log_model(
            model=model,
            name=MODEL_NAME,
            signature=signature,
        )

        print(f"Logged model URI: {model_info.model_uri}")
        return model_info


# Create production model
model_info = log_deployment_ready_model()

# register the model
mlflow.register_model(model_uri=model_info.model_uri, name=MODEL_NAME)


# To add an alias 'champion' to the latest model version created above
client = mlflow.MlflowClient()
latest_version = client.get_registered_model(name=MODEL_NAME).latest_versions[0] # this returns the latest model
print(f"Latest version: {latest_version.version}")
print(f"Model URI: {latest_version.source}")


client.set_registered_model_alias(
    name=MODEL_NAME,
    alias=ALIAS,
    version=latest_version.version
)

Started run with info:
<RunInfo: artifact_uri='mlflow-artifacts:/4/d295e2db675048acbf4a0953633be192/artifacts', end_time=None, experiment_id='4', lifecycle_stage='active', run_id='d295e2db675048acbf4a0953633be192', run_name='melodic-pig-373', start_time=1762187061001, status='RUNNING', user_id='joshuale'>
Logged model URI: models:/m-d3ca7275579a47bcbe72927e50e2df2d
🏃 View run melodic-pig-373 at: http://localhost:5000/#/experiments/4/runs/d295e2db675048acbf4a0953633be192
🧪 View experiment at: http://localhost:5000/#/experiments/4


Successfully registered model 'all-MiniLM-L12-v2'.
2025/11/04 00:24:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: all-MiniLM-L12-v2, version 1
Created version '1' of model 'all-MiniLM-L12-v2'.


Latest version: 1
Model URI: models:/m-d3ca7275579a47bcbe72927e50e2df2d
